In [10]:
from pathlib import Path

from tqdm import trange

from prompts import ANTIBODIES_CONTEXT_PROMPT, NAME_TO_AGENT
from run_scientific_meeting import run_scientific_meeting
from utils import load_summaries

In [9]:
# Set up the team
team_lead = "Principal Investigator"
team_members = tuple(NAME_TO_AGENT.keys())

# Set up key parameters
num_iterations = 3
num_rounds = 3
save_dir = Path("antibody_design")
model = "gpt-4o"
discussion_paths = []
contexts = (ANTIBODIES_CONTEXT_PROMPT,)

In [3]:
ANTIBODIES_PROMPT = "You are working on a project to develop antibodies for the SARS-CoV-2 spike protein, ideally for the newest variant of the virus and with broad spectrum activity across variants. Please design a specific antibody discovery approach for this target that uses machine learning to design antibody candidates. Decide what specific machine learning model to use and precisely how it will be used. If the model needs to be trained, please decide on a specific dataset for training. If the model is pre-trained, please explain exactly how it will be used for identifying or designing antibody candidates."

In [4]:
ANTIBODIES_QUESTIONS = (
    "Will you design the antibodies de novo or will you modify existing antibodies?",
    "Will you train a model from scratch or use a pre-trained model?",
    "What specific model architecture will you use?",
    "If training a model from scratch, what dataset will you use for training?",
    "How exactly will you use your model to design antibodies?"
)

In [5]:
# Run antibodies project design
for iteration_num in trange(num_iterations, desc="Project Iterations"):
    run_scientific_meeting(
        team_lead=team_lead,
        team_members=team_members,
        agenda=ANTIBODIES_PROMPT,
        agenda_questions=ANTIBODIES_QUESTIONS,
        contexts=contexts,
        save_dir=save_dir / "project_design",
        save_name=f"discussion_{iteration_num + 1}",
        num_rounds=num_rounds,
        model=model,
    )

Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:07<00:39,  7.96s/it]

Team Members:  33%|███▎      | 2/6 [00:15<00:31,  7.86s/it]

Team Members:  50%|█████     | 3/6 [00:23<00:23,  7.97s/it]

Team Members:  67%|██████▋   | 4/6 [00:29<00:14,  7.25s/it]

Team Members:  83%|████████▎ | 5/6 [00:37<00:07,  7.48s/it]

Team Members: 100%|██████████| 6/6 [00:47<00:00,  7.87s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [00:47<02:21, 47.20s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:17<01:29, 18.00s/it]

Team Members:  33%|███▎      | 2/6 [00:32<01:04, 16.16s/it]

Team Members:  50%|█████     | 3/6 [00:42<00:39, 13.21s/it]

Team Members:  67%|██████▋   | 4/6 [00:59<00:29, 14.72s/it]

Team Members:  83%|████████▎ | 5/6 [01:11<00:13, 13.69s/it]

Team Members: 100%|██████████| 6/6 [01:24<00:00, 14.05s/it]

Rounds (+ Summary Round)

Input token count: 100,102
Output token count: 11,113
Max token length: 12,294
Cost: $0.67



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:11<00:56, 11.33s/it]

Team Members:  33%|███▎      | 2/6 [00:26<00:53, 13.33s/it]

Team Members:  50%|█████     | 3/6 [00:40<00:41, 13.67s/it]

Team Members:  67%|██████▋   | 4/6 [00:54<00:27, 13.76s/it]

Team Members:  83%|████████▎ | 5/6 [01:08<00:13, 13.85s/it]

Team Members: 100%|██████████| 6/6 [01:23<00:00, 13.92s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:23<04:10, 83.52s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:17<01:26, 17.24s/it]

Team Members:  33%|███▎      | 2/6 [00:28<00:55, 13.83s/it]

Team Members:  50%|█████     | 3/6 [00:40<00:38, 12.80s/it]

Team Members:  67%|██████▋   | 4/6 [00:55<00:27, 13.65s/it]

Team Members:  83%|████████▎ | 5/6 [01:12<00:14, 14.80s/it]

Team Members: 100%|██████████| 6/6 [01:35<00:00, 15.98s/it]

Rounds (+ Summary Round

Input token count: 144,381
Output token count: 16,264
Max token length: 17,445
Cost: $0.97



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:08<00:43,  8.78s/it]

Team Members:  33%|███▎      | 2/6 [00:16<00:31,  7.95s/it]

Team Members:  50%|█████     | 3/6 [00:24<00:24,  8.11s/it]

Team Members:  67%|██████▋   | 4/6 [00:33<00:17,  8.59s/it]

Team Members:  83%|████████▎ | 5/6 [00:46<00:09,  9.96s/it]

Team Members: 100%|██████████| 6/6 [00:57<00:00,  9.50s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [00:57<02:51, 57.02s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:12<01:04, 13.00s/it]

Team Members:  33%|███▎      | 2/6 [00:25<00:50, 12.52s/it]

Team Members:  50%|█████     | 3/6 [00:40<00:41, 13.99s/it]

Team Members:  67%|██████▋   | 4/6 [00:54<00:27, 13.83s/it]

Team Members:  83%|████████▎ | 5/6 [01:16<00:16, 16.90s/it]

Team Members: 100%|██████████| 6/6 [01:33<00:00, 15.60s/it]

Rounds (+ Summary Round

Input token count: 129,185
Output token count: 15,149
Max token length: 16,330
Cost: $0.87


In [6]:
ESM_PROMPT = "You are working on a project to develop antibodies for the SARS-CoV-2 spike protein, ideally for the newest variant of the virus and with broad spectrum activity across variants. You will use the ESM family of models for antibody design. Please design a method for applying ESM to this antibody design problem. Specify the exact model you will use and how you will use it to design antibodies."

In [7]:
ESM_QUESTIONS = (
    "Which ESM model will you use?"
    "Will you design the antibodies de novo or will you modify existing antibodies?",
    "How exactly will you use your model to design antibodies?",
    "What is the precise process for designing antibodies and selecting candidates for experimental validation?",
    "How will you computationally evaluate the quality of the design antibodies?",
    "What objectives will you optimize for in the design process?"
)

In [8]:
# Run ESM project design
for iteration_num in trange(num_iterations, desc="Project Iterations"):
    run_scientific_meeting(
        team_lead=team_lead,
        team_members=team_members,
        agenda=ESM_PROMPT,
        agenda_questions=ESM_QUESTIONS,
        contexts=contexts,
        save_dir=save_dir / "esm",
        save_name=f"discussion_{iteration_num + 1}",
        num_rounds=num_rounds,
        model=model,
    )

Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:07<00:37,  7.43s/it]

Team Members:  33%|███▎      | 2/6 [00:15<00:31,  7.90s/it]

Team Members:  50%|█████     | 3/6 [00:26<00:27,  9.06s/it]

Team Members:  67%|██████▋   | 4/6 [00:39<00:21, 10.58s/it]

Team Members:  83%|████████▎ | 5/6 [00:51<00:11, 11.20s/it]

Team Members: 100%|██████████| 6/6 [01:02<00:00, 10.48s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:02<03:08, 62.90s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:09<00:46,  9.39s/it]

Team Members:  33%|███▎      | 2/6 [00:28<01:00, 15.12s/it]

Team Members:  50%|█████     | 3/6 [00:41<00:41, 13.93s/it]

Team Members:  67%|██████▋   | 4/6 [00:51<00:25, 12.63s/it]

Team Members:  83%|████████▎ | 5/6 [01:03<00:12, 12.33s/it]

Team Members: 100%|██████████| 6/6 [01:22<00:00, 13.79s/it]

Rounds (+ Summary Round)

Input token count: 116,197
Output token count: 12,655
Max token length: 13,796
Cost: $0.77



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:15<01:19, 15.86s/it]

Team Members:  33%|███▎      | 2/6 [00:26<00:51, 12.79s/it]

Team Members:  50%|█████     | 3/6 [00:36<00:34, 11.58s/it]

Team Members:  67%|██████▋   | 4/6 [00:47<00:22, 11.46s/it]

Team Members:  83%|████████▎ | 5/6 [01:13<00:16, 16.37s/it]

Team Members: 100%|██████████| 6/6 [01:23<00:00, 13.98s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:23<04:11, 83.85s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:23<01:58, 23.66s/it]

Team Members:  33%|███▎      | 2/6 [00:31<00:56, 14.13s/it]

Team Members:  50%|█████     | 3/6 [00:40<00:35, 11.99s/it]

Team Members:  67%|██████▋   | 4/6 [00:52<00:23, 11.90s/it]

Team Members:  83%|████████▎ | 5/6 [01:04<00:12, 12.12s/it]

Team Members: 100%|██████████| 6/6 [01:21<00:00, 13.57s/it]

Rounds (+ Summary Round

Input token count: 117,639
Output token count: 12,087
Max token length: 13,228
Cost: $0.77



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:08<00:43,  8.60s/it]

Team Members:  33%|███▎      | 2/6 [00:20<00:41, 10.32s/it]

Team Members:  50%|█████     | 3/6 [00:30<00:30, 10.33s/it]

Team Members:  67%|██████▋   | 4/6 [01:00<00:36, 18.26s/it]

Team Members:  83%|████████▎ | 5/6 [01:12<00:15, 15.87s/it]

Team Members: 100%|██████████| 6/6 [01:26<00:00, 14.36s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:26<04:18, 86.19s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:14<01:11, 14.33s/it]

Team Members:  33%|███▎      | 2/6 [00:22<00:42, 10.66s/it]

Team Members:  50%|█████     | 3/6 [00:33<00:32, 10.94s/it]

Team Members:  67%|██████▋   | 4/6 [00:41<00:19,  9.84s/it]

Team Members:  83%|████████▎ | 5/6 [00:52<00:10, 10.15s/it]

Team Members: 100%|██████████| 6/6 [01:04<00:00, 10.69s/it]

Rounds (+ Summary Round

Input token count: 113,878
Output token count: 12,845
Max token length: 13,986
Cost: $0.76


In [11]:
# Select preferred summary
discussion_paths.append(save_dir / "esm" / "discussion_3.json")

In [12]:
# Load summaries
summaries = load_summaries(discussion_paths=discussion_paths)

In [13]:
ESM_IMPLEMENT_PROMPT = "You now need to implement an ESM-based approach for antibody design. Please write code to implement the method you designed in the previous discussion. You should include the following components: 1. Loading the ESM model, 2. Designing antibodies using the ESM model, 3. Evaluating the designed antibodies computationally, 4. Selecting the best candidates for experimental validation. Please provide a detailed description of each component and the code to implement it."

In [14]:
ESM_IMPLEMENT_QUESTIONS = (
    "How will you load the ESM model?",
    "How will you design antibodies using the ESM model?",
    "How will you evaluate the designed antibodies computationally?",
    "How will you select the best candidates for experimental validation?"
)

In [15]:
# Run ESM implement
for iteration_num in trange(num_iterations, desc="Project Iterations"):
    run_scientific_meeting(
        team_lead=team_lead,
        team_members=team_members,
        agenda=ESM_IMPLEMENT_PROMPT,
        agenda_questions=ESM_IMPLEMENT_QUESTIONS,
        summaries=summaries,
        contexts=contexts,
        save_dir=save_dir / "esm_implement",
        save_name=f"discussion_{iteration_num + 1}",
        num_rounds=num_rounds,
        model=model,
    )

Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:29<02:27, 29.51s/it]

Team Members:  33%|███▎      | 2/6 [00:41<01:15, 18.99s/it]

Team Members:  50%|█████     | 3/6 [00:50<00:44, 14.81s/it]

Team Members:  67%|██████▋   | 4/6 [01:04<00:28, 14.25s/it]

Team Members:  83%|████████▎ | 5/6 [01:14<00:12, 12.94s/it]

Team Members: 100%|██████████| 6/6 [01:27<00:00, 14.66s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:27<04:23, 87.94s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [01:00<05:02, 60.50s/it]

Team Members:  33%|███▎      | 2/6 [01:08<01:59, 29.79s/it]

Team Members:  50%|█████     | 3/6 [01:18<01:02, 20.69s/it]

Team Members:  67%|██████▋   | 4/6 [01:30<00:34, 17.05s/it]

Team Members:  83%|████████▎ | 5/6 [01:44<00:16, 16.07s/it]

Team Members: 100%|██████████| 6/6 [02:01<00:00, 20.33s/it]

Rounds (+ Summary Round)

Input token count: 203,258
Output token count: 17,910
Max token length: 20,067
Cost: $1.28



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:04<00:23,  4.68s/it]

Team Members:  33%|███▎      | 2/6 [00:12<00:26,  6.63s/it]

Team Members:  50%|█████     | 3/6 [00:25<00:27,  9.30s/it]

Team Members:  67%|██████▋   | 4/6 [00:33<00:18,  9.11s/it]

Team Members:  83%|████████▎ | 5/6 [00:42<00:08,  8.87s/it]

Team Members: 100%|██████████| 6/6 [00:51<00:00,  8.63s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [00:51<02:35, 51.78s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:12<01:00, 12.05s/it]

Team Members:  33%|███▎      | 2/6 [00:21<00:42, 10.53s/it]

Team Members:  50%|█████     | 3/6 [00:32<00:31, 10.63s/it]

Team Members:  67%|██████▋   | 4/6 [00:49<00:26, 13.27s/it]

Team Members:  83%|████████▎ | 5/6 [01:11<00:16, 16.28s/it]

Team Members: 100%|██████████| 6/6 [01:20<00:00, 13.49s/it]

Rounds (+ Summary Round

Input token count: 144,603
Output token count: 15,113
Max token length: 17,270
Cost: $0.95



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:39<03:18, 39.68s/it]

Team Members:  33%|███▎      | 2/6 [00:50<01:31, 22.96s/it]

Team Members:  50%|█████     | 3/6 [01:00<00:49, 16.66s/it]

Team Members:  67%|██████▋   | 4/6 [01:07<00:26, 13.02s/it]

Team Members:  83%|████████▎ | 5/6 [01:17<00:11, 11.85s/it]

Team Members: 100%|██████████| 6/6 [01:41<00:00, 16.84s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [01:41<05:03, 101.02s/it]

Team Members:   0%|          | 0/6 [00:00<?, ?it/s]

Team Members:  17%|█▋        | 1/6 [00:14<01:11, 14.28s/it]

Team Members:  33%|███▎      | 2/6 [00:20<00:37,  9.46s/it]

Team Members:  50%|█████     | 3/6 [00:30<00:29,  9.86s/it]

Team Members:  67%|██████▋   | 4/6 [00:43<00:22, 11.14s/it]

Team Members:  83%|████████▎ | 5/6 [00:56<00:11, 11.63s/it]

Team Members: 100%|██████████| 6/6 [01:13<00:00, 12.32s/it]

Rounds (+ Summary Roun

Input token count: 136,860
Output token count: 13,587
Max token length: 15,744
Cost: $0.89
